In [34]:
import pandas as pd
from pandas import DataFrame
from pandas import Series

# Import dataset
services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)

# rename variables
# Are still missing the ones related to schedule
newColumns = {'_coordenadas_latitude': 'lat', 
              '_coordenadas_longitude' : 'long',  
              'pais' : 'ubicacion' , 
              'punto_reporte' : 'lugar_enc', 
              'ubicacion' : 'punto_ayud', 
              'cualotropunto' : 'punto_ayud_otro', 
              'infraestructura' : 'cuenta_con' , 
              'sexo' : 'identifica', 
              'organizacionpertence' : 'org_pert', 
              'infraestructuradiscapacidad' : 'serv_disca' , 
              'servicios' : 'serv_tipo',
              'siserviciosnna' : 'serv_dif_n',
              'serviciosnna' : 'cual_serv1',
              'otroservicionna' : 'otro_cual_',
              'siserviciosmujeres' : 'serv_dif_m',
              'serviciosmujeres' : 'cual_ser_2',
              'otroserviciomujeres' : 'otro_dif_m',
              }
services_carto = services.rename(columns=newColumns)

c:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [61]:
def codifyServices(value: str, values_dict: dict[str, int], otherValue:str):
    if(type(value) == float):
        return ""
    raw_values = value.split(" ")
    output = []
    for value in raw_values:
        try:      
            codedValue = values_dict[value]
            output.append(str(codedValue) )
        except Exception as e:
            output.append(otherValue)
            
    return "|".join(output)

def processColumn(dfColumn:Series, values_dict: dict[int, str], other_value: str):
    reversed_values_dict = dict( [( x[1],x[0])  for x in values_dict.items()])
    return dfColumn.apply(lambda x: codifyServices(x, reversed_values_dict, other_value))

def processMultValueColumns(df: DataFrame, columnObjectsList: list[dict]):
    """
    df: DataFrame object
    columnsObjectsList: list of column object
    columnObject: dictionary {"target_column": str, "output_column": str, values_dict: dict, other_value: str}

    return DataFrame Object
    """
    for columnObject in columnObjectsList:
        try:
            target_column = columnObject["target_column"]
            output_column = columnObject["output_column"]
            values_dict = columnObject["values_dict"]
            other_value = columnObject["other_value"]
            df[output_column] =  processColumn(df[target_column], values_dict, other_value)
        except Exception as e:
            print(e)
            continue
    return df

def exportToFile(df: DataFrame,fileType: str, exportName: str):
    """ 
    df -> Pandas DataFrame object
    fileType -> Either "csv" or "json"
    exportName -> File location
    """
    if(fileType == "csv"):
        name = f"{exportName}.csv"
        df.to_csv(name)
        print (f"data export to {name}")
    else:
        name = f"{exportName}.json"
        df.to_json(name, orient="records")
        print (f"data export to {name}")
    

In [59]:
# serv_tipo id the variable to test (separating by pipe symbol and categorized with number)

services_dict = {
  1 :"asesorialegal",
  2 :"alimentacionynutricion",
  3 :"saludprimerosauxiliosyatencionmedica",
  4 :"aguapotable",
  5 :"saneamiento",
  6 :"higiene",
  7 :"educacion",
  8 :"alojamientotemporal",
  9 :"serviciosdeproteccion",
  10 :"acompañamientoayudapsicosocial",
  11 :"transportehumanitario",
  12 :"restablecimientodecontactofamiliar",
  13:"transferencias",
  14:"Otro",
}

# re structure variable cuenta_con
cuenta_con_dict = {
  1 :"accesoaserviciodeinternet",
  2 :"accesoaservicioelectrico",
  3 :"areadecomedorescocina",
  4 :"areaparadisposiciondebasuras",
  5 :"areascomunes",
  6 :"areasdelavanderia",
  7 :"areasderegistrodelapoblacion",
  8 :"ningunadelasanteriores",
}

# re structure variable children services (cual_serv1)
cual_serv1_dict = {
  1 :"nnaalimentacionynutricion",
  2 :"nnaapoyooayudapsicosocial",
  3 :"nnabanosexclusivosnna",
  4 :"nnacuidadosalternativos",
  5 :"nnaentregadekits",
  6 :"espaciosamigables",
  7 :"nnagestiondecasos",
  8 :"nnaotro",
  9 : " ",
}

# re structure variable women services (cual_ser_2)
cual_ser_2_dict = {
  1 :"mujeresapoyonutricionalgestantesylactantes",
  2 :"mujerescontrolesprenatales",
  3 :"mujeresduchasobanos",
  4 :"mujeresespaciosseguros",
  5 :"mujeresgestiondecasosdeviolenciasbasadasengenero",
  6 :"mujeresproductosdehigienemenstrual",
  7 :"mujeresotro",
  8 : " ",
}

# re structure variable data storage (almacenamientoregistros)
registro_dict = {
  1 :"archivofisico",
  2 :"hojacalculo",
  3 :"software",
  4 :"otroalmacenarregistro",
  5 : " ",
}

values = [
    {
        "target_column": "serv_tipo",
        "output_column": "serv_tipo1",
        "values_dict": services_dict,
        "other_value": "0"
    },
    {
        "target_column": "cuenta_con",
        "output_column": "cuenta_c_1",
        "values_dict": cuenta_con_dict,
        "other_value": "0"
    },
    {
        "target_column": "cual_serv1",
        "output_column": "cual_ser_1",
        "values_dict": cual_serv1_dict,
        "other_value": "0"
    },
    {
        "target_column": "cual_ser_2",
        "output_column": "cual_ser_3",
        "values_dict": cual_ser_2_dict,
        "other_value": "0"
    },
    {
        "target_column": "almacenamientoregistros",
        "output_column": "almacenamientoregistros_",
        "values_dict": cual_ser_2_dict,
        "other_value": "0"
    },
]




In [65]:
output_df = processMultValueColumns(services_carto, values)
exportToFile(output_df, "csv", "output")

processing serv_tipo
processing cuenta_con
processing cual_serv1
processing cual_ser_2
processing almacenamientoregistros
data export to output.csv.csv
